<a href="https://colab.research.google.com/github/ethan1606/Plant_test1/blob/main/yolo_model_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Connect to GPU
select T4 GPU in runtime and connect

# 2. Upload dataset
upload labelled dataset into the folder as a data.zip file

Then unzip the file using the code below

In [ ]:
#unzip the data.zip file into a folder named 'dataset'
!unzip /content/data.zip -d /content/dataset

Note: the '!' allows shell commands to be run in the notebook rather than the terminal

# 3. Split the dataset into train and validation
To train a neural network the dataset must be split into training and validation images where during each epoch every image in the train folder is passed through the network performing backpropagation to adjust the weights and biases and then at the end of each epoch the images in the validation folder are used to check the model's performance

This will be done using a Python program from EdjeElectronics:

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# deploy the program specifiying where the dataset folder is and the split ratio of train to validation
!python train_val_split.py --datapath="/content/dataset" --train_pct=0.9

# 4. Install Ultralytics
Ultralytics must be installed to use the yolo models

In [ ]:
!pip install ultralytics

# 5. Reformat dataset for Ultralytics
Ultralytics requires a specific file structure of the dataset with a YAML file

Again using code from EdjeElectronics to create the data.yaml file:

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/dataset/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

#6 Train the YOLO model
Finally train with the chosen yolo model for a specified number of epochs

The best trained model will be saved to content/runs/detect/train/weights/best.pt and results can be found at content/runs/detect/train

In [ ]:
#specify data path, model to use, number of epochs, and image size
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

In [ ]:
!yolo detect val data=/content/data.yaml model=runs/detect/train/weights/best.pt

# 7. Test the model
Now the model is trained we can use the images from the validation folder to see how it peforms

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

# 8. Zip model
Zip the detect folder to download the model and the results

In [ ]:
# Create "model" folder to store model and results
!mkdir /content/model
!cp -r /content/runs/detect /content/model

# Zip into "model.zip"
%cd model
!zip -r /content/model.zip detect
%cd /content